https://towardsdatascience.com/the-k-prototype-as-clustering-algorithm-for-mixed-data-type-categorical-and-numerical-fe7c50538ebb

# Import Libraries 

In [2]:
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
!pip install plotnine
!pip install kmodes

from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
# Import module for data manipulation
import pandas as pd
# Import datetime
import datetime as dt 
# Import module for linear algebra
import numpy as np
# Import module for data visualization
from plotnine import *
import plotnine
import matplotlib 
import matplotlib.pyplot as plt
# Import module for k-protoype cluster
from kmodes.kprototypes import KPrototypes
# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)
# Format scientific notation from Pandas
pd.set_option('display.float_format', lambda x: '%.3f' % x)

  Using cached plotnine-0.12.1-py3-none-any.whl (1.3 MB)
  Using cached statsmodels-0.14.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.1 MB)
  Using cached mizani-0.9.2-py3-none-any.whl (73 kB)
  Using cached pandas-2.0.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Using cached patsy-0.5.3-py2.py3-none-any.whl (233 kB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.2.5
    Not uninstalling pandas at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'pandas'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [6]:
# Load the data
df = pd.read_csv(r'clienttable.csv')
# The dimension of data
print('Dimension data: {} rows and {} columns'.format(len(df), len(df.columns)))
# Print the first 5 rows
df.head()




Dimension data: 38912 rows and 26 columns


,client_id,country,state,city,device,brand_name,model_name,operating_system,operating_system_version,total_event_count,...,total_contactus_form_submit,total_engagement_time,engaged_sessions,avg_events_per_session,formatted_first_touch,formatted_engagement_time,avg_pageviews_per_session,engagement_rate,bounces,bounce_rate
0,NaN,United States,Wisconsin,Washington,mobile,OnePlus,iPhone,iOS,iOS 16.5,72,...,0,211.149,4,1.900,5/18/2023 11:23,0:03:31,1.180,12.12%,29,87.88%
1,1000008371.000,United States,Minnesota,Orono,desktop,Apple,Safari,Macintosh,Macintosh Intel 13.0,5,...,0,NaN,0,1.667,5/19/2023 18:18,NaN,0.670,0%,3,100%
2,1000014774.000,United States,California,Garden Grove,mobile,Coolpad,CP3320AS2,Android,Android 10.0.0,4,...,0,0.028,0,4.000,6/7/2023 2:39,0:00:00,3.000,0%,1,100%
3,1000019378.000,United States,Oregon,Boardman,desktop,Apple,Safari,Macintosh,Macintosh Intel 10.14,4,...,0,0.004,0,4.000,6/1/2023 20:22,0:00:00,3.000,0%,1,100%
4,1000020437.000,United States,California,Shasta Lake,mobile,Apple,iPhone,iOS,iOS 16.5,3,...,0,NaN,0,3.000,6/11/2023 1:54,NaN,2.000,0%,1,100%


# Data Inspection

In [4]:
# Inspect the data type
df.info()
# Inspect the categorical variables
df.select_dtypes('object').nunique()
# Inspect the numerical variables
df.describe()
# Check missing value
df.isna().sum()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43995 entries, 0 to 43994
Data columns (total 22 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   client_id                                   43994 non-null  float64
 1   country                                     43918 non-null  object 
 2   state                                       43463 non-null  object 
 3   city                                        41510 non-null  object 
 4   device                                      43995 non-null  object 
 5   brand_name                                  43771 non-null  object 
 6   model_name                                  43771 non-null  object 
 7   operating_system                            43986 non-null  object 
 8   operating_system_version                    43995 non-null  object 
 9   total_event_count                           43995 non-null  int64  
 10  first_touc

,client_id,total_event_count,total_sessions,total_pageviews,total_purchases,total_trade_lead_form_submit,total_residentialconsumer_lead_form_submit,total_contactus_form_submit,avg_engagement_time,avg_events_per_session,avg_engagement_time_hours,avg_pageviews_per_session
count,43994.000,43995.000,43995.000,43995.000,43995.000,43995.000,43995.000,43995.000,0.000,43887.000,0.000,43887.000
mean,1006116910.397,6.353,1.389,4.954,0.002,0.004,0.001,0.002,NaN,4.101,NaN,3.212
std,303899134.922,16.339,1.482,15.487,0.049,0.121,0.043,0.073,NaN,9.432,NaN,9.545
min,102259.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN,1.000,NaN,0.000
25%,1035599314.000,2.000,1.000,1.000,0.000,0.000,0.000,0.000,NaN,2.000,NaN,1.000
50%,1093493489.500,3.000,1.000,2.000,0.000,0.000,0.000,0.000,NaN,3.000,NaN,2.000
75%,1150938892.500,5.000,1.000,4.000,0.000,0.000,0.000,0.000,NaN,4.000,NaN,3.000
max,1207665453.000,1664.000,72.000,1663.000,2.000,10.000,5.000,8.000,NaN,1664.000,NaN,1663.000


# EDA/Processing

In [4]:
df_country = df.groupby('country').agg({
    'country': 'count',
    'total_event_count': 'mean',
    'total_sessions': 'mean',
    'total_pageviews': 'mean',
    'total_purchases': 'mean',
    'total_trade_lead_form_submit': 'mean',
    'total_residentialconsumer_lead_form_submit': 'mean',
    'total_contactus_form_submit': 'mean',
    'total_engagement_time': 'mean',
    'engaged_sessions': 'mean',
    'bounces': 'mean',
    'avg_pageviews_per_session': 'mean',   
}).rename(columns={'country': 'total'}).reset_index().sort_values('total', ascending=True)

#Calculated Fields
df_country['bounce_rate'] = df_country['bounces']/df_country['total_sessions']
df_country['avg_pageviews_per_session'] = df_country['total_pageviews']/df_country['total_sessions']
df_country['engagement_rate'] = df_country['engaged_sessions']/df_country['total_sessions']
df_country['converted_engagement_time'] = pd.to_datetime(df_country['total_engagement_time'], unit='s').dt.strftime("%H:%M:%S")
df_country

In [13]:
df_state = pd.DataFrame(df['state'].value_counts()).reset_index()
df_state.rename(columns={'state': 'total'}, inplace=True)
df_state = df_state.sort_values('total', ascending=True).reset_index(drop=True)
df_state = df.groupby(['country','state']).agg({
    'state': 'count',
    'total_event_count': 'mean',
    'total_sessions': 'mean',
    'total_pageviews': 'mean',
    'total_purchases': 'mean',
    'total_trade_lead_form_submit' : 'mean',
    'total_residentialconsumer_lead_form_submit' : 'mean',
    'total_contactus_form_submit' : 'mean',
    'total_engagement_time' : 'mean',
    'engaged_sessions' : 'mean',
    #'formatted_engagement_time' : 'mean',
    'avg_pageviews_per_session' : 'mean',
    #'engagement_rate' : 'mean',
    'bounces' : 'mean',
    #'bounce_rate' : 'mean'
}).rename(columns={'state': 'total'}).reset_index().sort_values('total', ascending=True)
df_state.rename(columns={'state': 'total', 'country': 'country'}, inplace=True)

#Calculated Fields
df_state['bounce_rate'] = df_country['bounces']/df_country['total_sessions']
df_state['avg_pageviews_per_session'] = df_country['total_pageviews']/df_country['total_sessions']
df_state['engagement_rate'] = df_country['engaged_sessions']/df_country['total_sessions']
df_state['converted_engagement_time'] = pd.to_datetime(df_country['total_engagement_time'], unit='s').dt.strftime("%H:%M:%S")

df_state

In [ ]:
df_ops_mean = df.groupby('operating_system').agg({
    'device': 'count',
    'total_event_count': 'mean',
    'total_sessions': 'mean',
    'total_pageviews': 'mean',
    'total_purchases': 'mean',
    'total_trade_lead_form_submit' : 'mean',
    'total_residentialconsumer_lead_form_submit' : 'mean',
    'total_contactus_form_submit' : 'mean',   
    'total_engagement_time' : 'mean',
    'engaged_sessions' : 'mean',
    'avg_pageviews_per_session' : 'mean',
    'bounces': 'mean',
}).rename(columns={'device': 'Total'}).reset_index().sort_values('Total', ascending=True)

df_ops_mean.rename(columns={'device': 'Total'}, inplace=True)
df_ops_mean.sort_values('Total', ascending=False, inplace=True)
#Calculated Fields
df_ops_mean['bounce_rate'] = df_country['bounces']/df_country['total_sessions']
df_ops_mean['avg_pageviews_per_session'] = df_country['total_pageviews']/df_country['total_sessions']
df_ops_mean['engagement_rate'] = df_country['engaged_sessions']/df_country['total_sessions']
df_ops_mean['converted_engagement_time'] = pd.to_datetime(df_country['total_engagement_time'], unit='s').dt.strftime("%H:%M:%S")

df_ops_mean


OPS totals

In [ ]:
df_ops = pd.DataFrame(df['device'].value_counts()).reset_index()
df_ops.rename(columns = {'index':'device', 'device':'Total'}, inplace = True)
df_region = df_ops.sort_values('Total', ascending = True).reset_index(drop = True)
# The dataframe
df_ops = df.groupby('operating_system').agg({
    'device': 'count',
    'total_event_count': 'mean',
    'total_sessions': 'mean',
    'total_pageviews': 'mean',
    'total_purchases': 'mean',
   'total_trade_lead_form_submit' : 'mean',
    'total_residentialconsumer_lead_form_submit' : 'mean',
    'total_contactus_form_submit' : 'mean',   
    'total_engagement_time' : 'mean',
    'engaged_sessions' : 'mean',
    'avg_pageviews_per_session' : 'mean',
    'bounces': 'mean',

}).rename(columns={'device': 'total'}).reset_index().sort_values('total', ascending=True)
#Calculated Fields
df_ops['bounce_rate'] = df_country['bounces']/df_country['total_sessions']
df_ops['avg_pageviews_per_session'] = df_country['total_pageviews']/df_country['total_sessions']
df_ops['engagement_rate'] = df_country['engaged_sessions']/df_country['total_sessions']
df_ops['converted_engagement_time'] = pd.to_datetime(df_country['total_engagement_time'], unit='s').dt.strftime("%H:%M:%S")

df_ops

In [ ]:
# Rest of your code remains the same
sortedops_df = df_ops_mean.sort_values(by='total_event_count', ascending=False)

# Select the top three rows based on the sorted metric
top_three_ops = sortedops_df.head(3)

# Extract the required data
operating_systems = top_three_ops['operating_system']
total_event_count = top_three_ops['total_event_count']
total_pageviews = top_three_ops['total_pageviews']
total_sessions = top_three_ops['total_sessions']
total_purchases = top_three_ops['total_purchases']

# Set the positions for the bars on the x-axis
x = np.arange(len(operating_systems))

width = 0.2

fig, ax = plt.subplots()

# Plot the bars for each metric
ax.bar(x - width, total_event_count, width, label='Total Event Count', color='#981220', alpha=0.6)
ax.bar(x, total_pageviews, width, label='Total Pageviews', color='#80797c', alpha=0.6)
ax.bar(x + width, total_sessions, width, label='Total Sessions', color='#5b9bd5', alpha=0.6)
ax.bar(x + (2 * width), total_purchases, width, label='Total Purchases', color='#c95f5f', alpha=0.6)

# Set the x-axis tick positions and labels
ax.set_xticks(x)
ax.set_xticklabels(operating_systems)

# Set the y-axis label
ax.set_ylabel('Count')

# Set the chart title
ax.set_title('AVG OPS Metrics (Top Three)')

# Add a legend
ax.legend()

# Show the chart
plt.show()

In [ ]:
# Extract the required data
operating_systems = df_ops_mean['operating_system']
total_trade_lead_form_submit = df_ops_mean['total_trade_lead_form_submit']
total_residentialconsumer_lead_form_submit = df_ops_mean['total_residentialconsumer_lead_form_submit']
total_contactus_form_submit = df_ops_mean['total_contactus_form_submit']

# Set the positions for the bars on the x-axis
x = np.arange(len(operating_systems))

width = 0.2

fig, ax = plt.subplots()

# Plot the bars for each metric
ax.bar(x - width, total_trade_lead_form_submit, width, label='total_trade_lead_form_submit', color='#981220', alpha=0.6)
ax.bar(x, total_residentialconsumer_lead_form_submit, width, label='total_residentialconsumer_lead_form_submit', color='#80797c', alpha=0.6)
ax.bar(x + width, total_contactus_form_submit, width, label='total_contactus_form_submit', color='#5b9bd5', alpha=0.6)

# Set the x-axis tick positions and labels
ax.set_xticks(x)
ax.set_xticklabels(operating_systems)

# Set the y-axis label
ax.set_ylabel('Count')

# Set the chart title
ax.set_title('AVG OPS Form Submissions ')

# Add a legend
ax.legend()

# Show the chart
plt.show()


In [ ]:
df_device = pd.DataFrame(df['device'].value_counts()).reset_index()
df_device.rename(columns = {'index':'device', 'device':'Total'}, inplace = True)
df_device = df_device.sort_values('Total', ascending = True).reset_index(drop = True)
# The dataframe
df_device = df.groupby('device').agg({
    'device': 'count',
    'total_event_count': 'sum',
    'total_sessions': 'sum',
    'total_pageviews': 'sum',
    'total_purchases': 'sum',

}).rename(columns={'device': 'total'}).reset_index().sort_values('total', ascending=True)

df_device

Dev means

In [ ]:
df_device_mean = pd.DataFrame(df['device'].value_counts()).reset_index()
df_device_mean.rename(columns = {'index':'device', 'device':'Total'}, inplace = True)
df_device_mean = df_device_mean.sort_values('Total', ascending = True).reset_index(drop = True)
# The dataframe
df_device_mean = df.groupby('device').agg({
    'device': 'count',
    'total_event_count': 'mean',
    'total_sessions': 'mean',
    'total_pageviews': 'mean',
    'total_purchases': 'mean',

}).rename(columns={'device': 'total'}).reset_index().sort_values('total', ascending=True)

df_device_mean

In [ ]:
device_names = df_device_mean['device']
total_event_count = df_device_mean['total_event_count']
total_pageviews = df_device_mean['total_pageviews']
total_sessions = df_device_mean['total_sessions']
total_purchases = df_device_mean['total_purchases']

x = np.arange(len(device_names))

width = 0.2

fig, ax = plt.subplots()

ax.bar(x - width, total_event_count, width, label='Total Event Count', color='#981220', alpha=0.6)
ax.bar(x, total_pageviews, width, label='Total Pageviews', color='#80797c', alpha=0.6)
ax.bar(x + width, total_sessions, width, label='Total Sessions', color='#5b9bd5', alpha=0.6)
ax.bar(x + (2 * width), total_purchases, width, label='Total Purchases', color='#c95f5f', alpha=0.6)

ax.set_xticks(x)
ax.set_xticklabels(device_names)

ax.set_ylabel('Count')
ax.set_title('AVG Mobile Device Metrics')

ax.legend()

plt.show()


## brand name

In [ ]:
df_brand_name = df.groupby('brand_name').agg({
    'brand_name': 'count',
    'total_event_count': 'sum',
    'total_sessions': 'sum',
    'total_pageviews': 'sum',
    'total_purchases': 'sum',  
    'total_trade_lead_form_submit' : 'mean',
    'total_residentialconsumer_lead_form_submit' : 'mean',
    'total_contactus_form_submit' : 'mean',
    'total_engagement_time' : 'mean',
    'engaged_sessions' : 'mean',
    #'formatted_engagement_time' : 'mean',
    'avg_pageviews_per_session' : 'mean',
    #'engagement_rate' : 'mean',
    'bounces' : 'mean',
    #'bounce_rate' : 'mean'
}).rename(columns={'brand_name': 'Total'}).reset_index().sort_values('Total', ascending=True)
#Calculated Fields
df_brand_name['bounce_rate'] = df_country['bounces']/df_country['total_sessions']
df_brand_name['avg_pageviews_per_session'] = df_country['total_pageviews']/df_country['total_sessions']
df_brand_name['engagement_rate'] = df_country['engaged_sessions']/df_country['total_sessions']
df_brand_name['converted_engagement_time'] = pd.to_datetime(df_country['total_engagement_time'], unit='s').dt.strftime("%H:%M:%S")

df_brand_name

In [ ]:
df_brand_name_mean = df.groupby('brand_name').agg({
    'brand_name': 'count',
    'total_event_count': 'mean',
    'total_sessions': 'mean',
    'total_pageviews': 'mean',
    'total_purchases': 'mean',
    'total_trade_lead_form_submit' : 'mean',
    'total_residentialconsumer_lead_form_submit' : 'mean',
    'total_contactus_form_submit' : 'mean',
    'total_engagement_time' : 'mean',
    'engaged_sessions' : 'mean',
    #'formatted_engagement_time' : 'mean',
    'avg_pageviews_per_session' : 'mean',
    #'engagement_rate' : 'mean',
    'bounces' : 'mean',
    #'bounce_rate' : 'mean'
}).rename(columns={'brand_name': 'Total'}).reset_index().sort_values('Total', ascending=True)
#Calculated Fields
df_brand_name_mean['bounce_rate'] = df_country['bounces']/df_country['total_sessions']
df_brand_name_mean['avg_pageviews_per_session'] = df_country['total_pageviews']/df_country['total_sessions']
df_brand_name_mean['engagement_rate'] = df_country['engaged_sessions']/df_country['total_sessions']
df_brand_name_mean['converted_engagement_time'] = pd.to_datetime(df_country['total_engagement_time'], unit='s').dt.strftime("%H:%M:%S")

df_brand_name_mean


## Model Name

In [ ]:
df_model_name_mean = df.groupby('model_name').agg({
    'model_name': 'count',
    'total_event_count': 'mean',
    'total_sessions': 'mean',
    'total_pageviews': 'mean',
    'total_purchases': 'mean',
    'total_trade_lead_form_submit' : 'mean',
    'total_residentialconsumer_lead_form_submit' : 'mean',
    'total_contactus_form_submit' : 'mean',
    'total_engagement_time' : 'mean',
    'engaged_sessions' : 'mean',
    #'formatted_engagement_time' : 'mean',
    'avg_pageviews_per_session' : 'mean',
    #'engagement_rate' : 'mean',
    'bounces' : 'mean',
    #'bounce_rate' : 'mean'
}).rename(columns={'model_name': 'total'}).reset_index().sort_values('total', ascending=True)
#Calculated Fields
df_model_name_mean['bounce_rate'] = df_country['bounces']/df_country['total_sessions']
df_model_name_mean['avg_pageviews_per_session'] = df_country['total_pageviews']/df_country['total_sessions']
df_model_name_mean['engagement_rate'] = df_country['engaged_sessions']/df_country['total_sessions']
df_model_name_mean['converted_engagement_time'] = pd.to_datetime(df_country['total_engagement_time'], unit='s').dt.strftime("%H:%M:%S")

df_model_name_mean

In [ ]:
sorted_modelname_df = df_model_name_mean.sort_values(by='total_event_count', ascending=False)

top_five_modelname = sorted_modelname_df.head(5)

model_names = top_five_modelname['model_name']
total_event_count = top_five_modelname['total_event_count']
total_pageviews = top_five_modelname['total_pageviews']
total_sessions = top_five_modelname['total_sessions']
total_purchases = top_five_modelname['total_purchases']

x = np.arange(len(model_names))

width = 0.2

fig, ax = plt.subplots()

ax.bar(x - width, total_event_count, width, label='Total Event Count', color='#981220', alpha=0.6)
ax.bar(x, total_pageviews, width, label='Total Pageviews', color='#80797c', alpha=0.6)
ax.bar(x + width, total_sessions, width, label='Total Sessions', color='#5b9bd5', alpha=0.6)
ax.bar(x + (2 * width), total_purchases, width, label='Total Purchases', color='#c95f5f', alpha=0.6)

ax.set_xticks(x)
ax.set_xticklabels(model_names)

ax.set_ylabel('Count')
ax.set_title('AVG Model Name Metrics (Top 5)')

ax.legend()

plt.show()


In [ ]:
df_model_name = df.groupby('model_name').agg({
    'model_name': 'count',
    'total_event_count': 'sum',
    'total_sessions': 'sum',
    'total_pageviews': 'sum',
    'total_purchases': 'sum',
    'total_trade_lead_form_submit' : 'sum',
    'total_residentialconsumer_lead_form_submit' : 'sum',
    'total_contactus_form_submit' : 'sum',
    'total_engagement_time' : 'sum',
    'engaged_sessions' : 'sum',
    #'formatted_engagement_time' : 'mean',
    'avg_pageviews_per_session' : 'sum',
    #'engagement_rate' : 'mean',
    'bounces' : 'sum',
    #'bounce_rate' : 'mean'
}).rename(columns={'model_name': 'total'}).reset_index().sort_values('total', ascending=True)
#Calculated Fields
df_model_name['bounce_rate'] = df_country['bounces']/df_country['total_sessions']
df_model_name['avg_pageviews_per_session'] = df_country['total_pageviews']/df_country['total_sessions']
df_model_name['engagement_rate'] = df_country['engaged_sessions']/df_country['total_sessions']
df_model_name['converted_engagement_time'] = pd.to_datetime(df_country['total_engagement_time'], unit='s').dt.strftime("%H:%M:%S")

df_model_name

In [ ]:
sortedbrandname_df = df_brand_name_mean.sort_values(by='total_event_count', ascending=False)

# Select the top three rows based on the sorted metric
top_five_brandname = sortedbrandname_df.head(5)

# Extract the required data
brand_names=top_five_brandname['brand_name']
total_event_count = top_five_brandname['total_event_count']
total_pageviews = top_five_brandname['total_pageviews']
total_sessions = top_five_brandname['total_sessions']
total_purchases = top_five_brandname['total_purchases']

# Set the positions for the bars on the x-axis
x = np.arange(len(brand_names))

# Set the width of each bar
width = 0.2

# Create the figure and axis objects
fig, ax = plt.subplots()

# Plot the bars for each metric
ax.bar(x - width, total_event_count, width, label='Total Event Count', color='#981220', alpha=0.6)
ax.bar(x, total_pageviews, width, label='Total Pageviews', color='#80797c', alpha=0.6)
ax.bar(x + width, total_sessions, width, label='Total Sessions', color='#5b9bd5', alpha=0.6)
ax.bar(x + (2 * width), total_purchases, width, label='Total Purchases', color='#c95f5f', alpha=0.6)

# Set the x-axis tick positions and labels
ax.set_xticks(x)
ax.set_xticklabels(brand_names)

# Set the y-axis label
ax.set_ylabel('Count')

# Set the chart title
ax.set_title('AVG Brand Name Metrics(top 5)')

# Add a legend
ax.legend()

# Show the chart
plt.show()

In [ ]:
df_brand_name = df.groupby('brand_name').agg({
    'brand_name': 'count',
    'total_event_count': 'mean',
    'total_sessions': 'mean',
    'total_pageviews': 'mean',
    'total_purchases': 'mean',
    'total_trade_lead_form_submit' : 'mean',
    'total_residentialconsumer_lead_form_submit' : 'mean',
    'total_contactus_form_submit' : 'mean',
    'total_engagement_time' : 'mean',
    'engaged_sessions' : 'mean',
    #'formatted_engagement_time' : 'mean',
    'avg_pageviews_per_session' : 'mean',
    #'engagement_rate' : 'mean',
    'bounces' : 'mean',
    #'bounce_rate' : 'mean'
}).rename(columns={'brand_name': 'Total'}).reset_index().sort_values('Total', ascending=True)
#Calculated Fields
df_brand_name['bounce_rate'] = df_country['bounces']/df_country['total_sessions']
df_brand_name['avg_pageviews_per_session'] = df_country['total_pageviews']/df_country['total_sessions']
df_brand_name['engagement_rate'] = df_country['engaged_sessions']/df_country['total_sessions']
df_brand_name['converted_engagement_time'] = pd.to_datetime(df_country['total_engagement_time'], unit='s').dt.strftime("%H:%M:%S")

df_brand_name

## total session Visuals

Total sessions by device type

In [ ]:
plotnine.options.figure_size = (8, 4.8)

(
    ggplot(data=df_device) +
    geom_bar(
        aes(x='device',
            y='total_sessions'),
        fill=np.where(df_device['device'] == 'mobile', '#981220', '#80797c'),
        stat='identity'
    ) +
    geom_text(
        aes(x='device',
            y='total_sessions',
            label=df_device['total_sessions'].astype(str)),
        size=10,
        nudge_y=6
    ) +
    labs(title='Total sessions by device type') +
    xlab('device') +
    ylab('Total Sessions') +
    scale_x_discrete(limits=df_device['device'].tolist()) +
    theme_minimal() +
    coord_flip()
)

avg Total sessions by operating system

In [ ]:
plotnine.options.figure_size = (8, 4.8)

(
    ggplot(data=df_ops) +
    geom_bar(
        aes(x='operating_system',
            y='total_sessions'),
        fill=np.where(df_ops['operating_system'] == 'iOS', '#981220', '#80797c'),
        stat='identity'
    ) +
    geom_text(
        aes(x='operating_system',
            y='total_sessions',
            label=df_ops['total_sessions'].astype(str)),
        size=10,
        nudge_y=6
    ) +
    labs(title='avg Total sessions by operating system') +
    xlab('Operating System') +
    ylab('total_sessions') +
    scale_x_discrete(limits=df_ops['operating_system'].tolist()) +
    theme_minimal() +
    coord_flip()
)

## total event visuals

avg Total events by device type

In [ ]:
plotnine.options.figure_size = (8, 4.8)

(
    ggplot(data=df_device) +
    geom_bar(
        aes(x='device',
            y='total_event_count'),
        fill=np.where(df_device['device'] == 'mobile', '#981220', '#80797c'),
        stat='identity'
    ) +
    geom_text(
        aes(x='device',
            y='total_event_count',
            label=df_device['total_event_count'].astype(str)),
        size=10,
        nudge_y=6
    ) +
    labs(title='avg Total events by device type') +
    xlab('device') +
    ylab('Total events') +
    scale_x_discrete(limits=df_device['device'].tolist()) +
    theme_minimal() +
    coord_flip()
)

avg pageviews by ops

In [ ]:
plotnine.options.figure_size = (8, 4.8)

(
    ggplot(data=df_ops) +
    geom_bar(
        aes(x='operating_system',
            y='total_event_count'),
        fill=np.where(df_ops['operating_system'] == 'iOS', '#981220', '#80797c'),
        stat='identity'
    ) +
    geom_text(
        aes(x='operating_system',
            y='total_event_count',
            label=df_ops['total_event_count'].astype(str)),
        size=10,
        nudge_y=6
    ) +
    labs(title='avg Total Events by operating system') +
    xlab('Operating System') +
    ylab('Total Event Count') +
    scale_x_discrete(limits=df_ops['operating_system'].tolist()) +
    theme_minimal() +
    coord_flip()
)

## Total pageviews

pageviews by device

In [ ]:
plotnine.options.figure_size = (8, 4.8)

(
    ggplot(data=df_device) +
    geom_bar(
        aes(x='device',
            y='total_pageviews'),
        fill=np.where(df_device['device'] == 'mobile', '#981220', '#80797c'),
        stat='identity'
    ) +
    geom_text(
        aes(x='device',
            y='total_pageviews',
            label=df_device['total_pageviews'].astype(str)),
        size=10,
        nudge_y=6
    ) +
    labs(title='avg Total pageviews by device type') +
    xlab('device') +
    ylab('Total Pageviews') +
    scale_x_discrete(limits=df_device['device'].tolist()) +
    theme_minimal() +
    coord_flip()
)

pageviews by ops

In [ ]:
import numpy as np
from plotnine import *

plotnine.options.figure_size = (8, 4.8)

(
    ggplot(data=df_ops) +
    geom_bar(
        aes(x='operating_system',
            y='total_pageviews'),
        fill=np.where(df_ops['operating_system'] == 'iOS', '#981220', '#80797c'),
        stat='identity'
    ) +
    geom_text(
        aes(x='operating_system',
            y='total_pageviews',
            label=df_ops['total_pageviews'].astype(str)),
        size=10,
        nudge_y=6
    ) +
    labs(title='avg Total Events by operating system') +
    xlab('Operating System') +
    ylab('Total Pageviews') +
    scale_x_discrete(limits=df_ops['operating_system'].tolist()) +
    theme_minimal() +
    coord_flip()
)

## Total Purchases

In [ ]:
plotnine.options.figure_size = (8, 4.8)

(
    ggplot(data=df_device) +
    geom_bar(
        aes(x='device',
            y='total_purchases'),
        fill=np.where(df_device['device'] == 'mobile', '#981220', '#80797c'),
        stat='identity'
    ) +
    geom_text(
        aes(x='device',
            y='total_purchases',
            label=df_device['total_purchases'].astype(str)),
        size=10,
        nudge_y=6
    ) +
    labs(title='avg Total purchases by device type') +
    xlab('device') +
    ylab('Total Purchases') +
    scale_x_discrete(limits=df_device['device'].tolist()) +
    theme_minimal() +
    coord_flip()
)

In [ ]:
plotnine.options.figure_size = (8, 4.8)

(
    ggplot(data=df_ops) +
    geom_bar(
        aes(x='operating_system',
            y='total_purchases'),
        fill=np.where(df_ops['operating_system'] == 'iOS', '#981220', '#80797c'),
        stat='identity'
    ) +
    geom_text(
        aes(x='operating_system',
            y='total_purchases',
            label=df_ops['total_pageviews'].astype(str)),
        size=10,
        nudge_y=6
    ) +
    labs(title='avg Total Events by operating system') +
    xlab('Operating System') +
    ylab('Total Purchases') +
    scale_x_discrete(limits=df_ops['operating_system'].tolist()) +
    theme_minimal() +
    coord_flip()
)

# Processing

In [23]:
# Get the position of categorical columns
catColumnsPos = [df.columns.get_loc(col) for col in list(df.select_dtypes('object').columns)]
print('Categorical columns           : {}'.format(list(df.select_dtypes('object').columns)))
print('Categorical columns position  : {}'.format(catColumnsPos))

# Feature Engineering 

# Elbow Method

In [7]:
# Read the data
data = pd.read_csv('clienttable.csv')

# identify the categorical and numerical columns
numeric_cols = data.select_dtypes(include=[np.number]).columns
categorical_cols = data.select_dtypes(include=['object']).columns

print('Numeric columns:')
for col in numeric_cols:
    print(f'Column name: {col}, Position: {df.columns.get_loc(col)}')

print('\nCategorical columns:')
for col in categorical_cols:
    print(f'Column name: {col}, Position: {df.columns.get_loc(col)}')

Numeric columns:
Column name: client_id, Position: 0
Column name: total_event_count, Position: 9
Column name: total_sessions, Position: 11
Column name: total_pageviews, Position: 12
Column name: total_purchases, Position: 13
Column name: total_trade_lead_form_submit, Position: 14
Column name: total_residentialconsumer_lead_form_submit, Position: 15
Column name: total_contactus_form_submit, Position: 16
Column name: total_engagement_time, Position: 17
Column name: engaged_sessions, Position: 18
Column name: avg_events_per_session, Position: 19
Column name: avg_pageviews_per_session, Position: 22
Column name: bounces, Position: 24

Categorical columns:
Column name: country, Position: 1
Column name: state, Position: 2
Column name: city, Position: 3
Column name: device, Position: 4
Column name: brand_name, Position: 5
Column name: model_name, Position: 6
Column name: operating_system, Position: 7
Column name: operating_system_version, Position: 8
Column name: first_touch, Position: 10
Colu

In [10]:
# Read the data
data = pd.read_csv('clienttable.csv')

# Convert percentage to float
data['engagement_rate'] = data['engagement_rate'].str.rstrip('%').astype('float') / 100.0
data['bounce_rate'] = data['bounce_rate'].str.rstrip('%').astype('float') / 100.0


# Select numerical and categorical features
numeric_cols = [#'total_event_count', 'total_sessions',
'avg_events_per_session',# 'total_pageviews',
 'total_purchases',
                'total_trade_lead_form_submit', 'total_residentialconsumer_lead_form_submit',
                'total_contactus_form_submit', 'total_engagement_time', 'engaged_sessions',
                'avg_pageviews_per_session',# 'bounces',
                 'engagement_rate', 'bounce_rate'
                ]

categorical_cols = ['country', 'state', #'city', 'device', 'brand_name',
                    'model_name',# 'operating_system',
                     'operating_system_version']

numeric_features = data[numeric_cols]
categorical_features = data[categorical_cols]

# Concatenate numerical and categorical features into one DataFrame
data_combined = pd.concat([numeric_features, categorical_features], axis=1)

# Drop rows with NaN values
data_combined.dropna(inplace=True)

# Find the column indices of categorical features
categorical_feature_indices = list(range(numeric_features.shape[1], data_combined.shape[1]))

# Initialize an empty list to store the cost values
cost = []

# Define the range of cluster numbers to try
min_clusters = 1
max_clusters = 10

# Perform k-prototypes clustering for different cluster numbers
for k in range(min_clusters, max_clusters + 1):
    kproto = KPrototypes(n_jobs = -1, n_clusters=k, init='cao', random_state = 0)  # 'Cao' is another initializer for k-prototypes
    clusters = kproto.fit_predict(data_combined, categorical=categorical_feature_indices)
    # Append the cost value to the list
    cost.append(kproto.cost_)

# Plotting the elbow curve
plt.plot(range(min_clusters, max_clusters + 1), cost, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cost')
plt.title('K-Prototypes Elbow Method')
plt.show()


KeyboardInterrupt: 

In [12]:
data = pd.read_csv('clienttable.csv')

numeric_features = data.iloc[:, [10,12,13,14,15,16,17,18,19 ]].values
categorical_features = data.iloc[:, [2,3,4,5,6,7,8,9]].values 
# numeric_features:  total_event_count,total_sessions,total_pageviews, total_purchases, total_trade_form.., total_residential_form..., total_contactus_form...,
# categorical_features: country, state, city, device, brand_name, model_name, operating system, system_version

# Initialize an empty list to store the cost values
cost = []

# Define the range of cluster numbers to try
min_clusters = 1
max_clusters = 10

# Perform k-prototypes clustering for different cluster numbers
for k in range(min_clusters, max_clusters + 1):

    kproto = KPrototypes(n_clusters=k)
    
    # Fit the model to the data
    clusters = kproto.fit_predict(numeric_features, categorical_features)
    
    # Append the cost value to the list
    cost.append(kproto.cost_)

# Plotting the elbow curve
plt.plot(range(min_clusters, max_clusters + 1), cost, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cost')
plt.title('K-Prototypes Elbow Method')
plt.show()



In [ ]:
import pandas as pd
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt

import pandas as pd
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('clienttable.csv')

# Select the numeric and categorical features separately
numeric_features = data.iloc[:, [10, 12, 13, 14, 15, 16, 17, 18, 19]].values
categorical_features = data[['country', 'state', 'city', 'device', 'brand_name', 'model_name', 'operating_system', 'operating_system_version']].values

# Define the range of cluster numbers to try
min_clusters = 1
max_clusters = 10

# Initialize an empty list to store the cost values
cost = []

# Perform k-prototypes clustering for different cluster numbers
for k in range(min_clusters, max_clusters + 1):
    # Create a KPrototypes instance with k clusters
    kproto = KPrototypes(n_clusters=k)

    # Fit the model to the data
    clusters = kproto.fit_predict(numeric_features, categorical_features)

    # Append the cost value to the list
    cost.append(kproto.cost_)

# Plot the elbow curve
plt.plot(range(min_clusters, max_clusters + 1), cost, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Cost')
plt.title('K-Prototypes Elbow Method')
plt.show()


In [ ]:
df.describe().T

# K Prototype 

# Apply Clusters to Pagetable

# EDA on Pagetable

# Apply Characteristics

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c07d89ac-5a96-494b-a3e5-c0b991068de9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>